# 3for2 IAQ Data from BMS

Clayton Miller
Feb 22, 2017

Extract data and analyze data according to Jovan's instructions:

```
This will be the airside dataset. This will show the operation of air and water components of 3for2 for periods of time when we used kits to measure performance in the space. These plots should have time on the x-axis and then respective quantities on the y-axis. If you have some other idea to show it please go ahead with it. Idea from my perspective here is to show dinamic response of the system and difference between various spaces.

I will use names in the horsebrokedown dashboard.

For the period December 29th and January 7th and February 10th to 16th in the Airside System I would like to show an operation of the ventilation supply and CO2 levels in the exhaust air by comparing following readings:

CO2 sensors: B10_Area3for2_A_Ahu1_DOAS_CO2Ex
                       B10_Area3for2_A_Ahu3_DOAS_CO2Ex
                       B10_Area3for2_A_Ahu4_DOAS_CO2Ex

Air system supply flow rates: B10_Area3for2_A_Ahu1_DOAS_FanSuEh_FISu
                                               B10_Area3for2_A_Ahu3_DOAS_FanSuEh_FISu
                                               B10_Area3for2_A_Ahu4_DOAS_FanSuEh_FISu

DOAS air system modulation: B10_Area3for2_A_Ahu1_DOAS_FanSuEh_MdltSu
                                                 B10_Area3for2_A_Ahu3_DOAS_FanSuEh_MdltSu
                                                 B10_Area3for2_A_Ahu4_DOAS_FanSuEh_MdltSu

For the period December 29th and January 7th and February 10th to 16th in the Waterside System

Valve positions Chilled Beams: B10_Area3for2_R_FO3_CcgVlvPos_1
                                                  B10_Area3for2_R_HRM_CcgVlvPos_1
                                                  B10_Area3for2_R_O_CcgVlvPos_1
                                                  B10_Area3for2_R_P_CcgVlvPos_1

Supply water temperatures PCB: B10_Area3for2_Mtr_CGrpPCB_FO3_TFI
                                                     B10_Area3for2_Mtr_CGrpPCB_HRM_TFI
                                                     B10_Area3for2_Mtr_CGrpPCB_Om1_TFI
                                                     B10_Area3for2_Mtr_CGrpPCB_Pm1_TFI

Water flowrate PCB: B10_Area3for2_Mtr_CGrpPCB_FO3_FL
B10_Area3for2_Mtr_CGrpPCB_HRM_FL
B10_Area3for2_Mtr_CGrpPCB_Om1_FL
B10_Area3for2_Mtr_CGrpPCB_Om2_FL
B10_Area3for2_Mtr_CGrpPCB_Om3_FL
B10_Area3for2_Mtr_CGrpPCB_Om4_FL
B10_Area3for2_Mtr_CGrpPCB_Om5_FL
B10_Area3for2_Mtr_CGrpPCB_Pm1_FL
B10_Area3for2_Mtr_CGrpPCB_Pm2_FL
```

In [62]:
import pandas as pd
from influxdb import DataFrameClient
import pytz
import seaborn as sns
import json
import datetime
import matplotlib.pyplot as plt

from pylab import *
from __future__ import division

In [63]:
%matplotlib inline

## First let's grab the BMS data

In [89]:
# host='horsebrokedown-clocktower-1.c.influxdb.com' 
# port='8086'
# login='influxdb'
# pw='a873671b790d4523'
# dbname = 'siemenstest'

host='lonepine-76a67992.influxcloud.net'
port='8086' 
login='loaddata'
pw='uwcsearocks88'
database = 'bms'

client = DataFrameClient(host, port, login, pw, database, ssl=True, verify_ssl=True)

In [65]:
seriesname = pd.DataFrame(list(client.query('SHOW SERIES'))[0])

In [66]:
series = seriesname['key'].str.split(",").apply(pd.Series)

In [67]:
series.head()

,0,1,2,3,4
0,bmspoint,DeviceNum=0,Location=DOAS1,System=Ventilation,Type=CO2
1,bmspoint,DeviceNum=0,Location=DOAS2,System=Ventilation,Type=CO2
2,bmspoint,DeviceNum=0,Location=DOAS3,System=Ventilation,Type=CO2
3,bmspoint,DeviceNum=0,Location=DOAS4,System=Ventilation,Type=CO2
4,bmspoint,DeviceNum=0,Location=FCU1,System=Ventilation,Type=CO2


In [68]:
series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 751 entries, 0 to 750
Data columns (total 5 columns):
0    751 non-null object
1    751 non-null object
2    751 non-null object
3    751 non-null object
4    751 non-null object
dtypes: object(5)
memory usage: 29.4+ KB


In [8]:
pointlist = list(series[0])

In [9]:
#client.query('SELECT * FROM bmspoint WHERE time > now() - 10d AND time < now() - 9d', chunked=True)

## Get all data from dates of Dec 29 - Jan 7

Dec 29 is 63 days ago -- we can pull out data 3 days at a time for the time range

In [78]:
dec_jan_timeranges = [(63,61),(61,59),(59,57),(57,55),(55,53)]
feb_timeranges = [(21,19),(19,17),(17,15),(15,13)]
febmar_timeranges = [(12,10),(10,8),(8,6),(6,5),(5,4),(4,3),(3,2),(2,1),(1,0)]

#dec_feb_timeranges = [(63,61),(61,59),(59,57),(57,55),(55,53),(21,19),(19,17),(17,15),(15,13)]
dec_feb_timeranges = [(73,71),(71,69),(69,67),(67,65),(65,63),(31,29),(29,27),(27,25),(25,23)] #had to increase by 10 as I picked up this notebook again after 10 days -- need a better way of getting the data

In [84]:
def get_data(begin, end, pointtype):
    data = client.query('SELECT * FROM bmspoint WHERE time > now() - '+str(end)+'d AND time < now() - '+str(begin)+'d')
    data = pd.DataFrame(data['bmspoint'])
    tz_pytz = pytz.timezone('Asia/Singapore')
    data = data.tz_convert(tz_pytz)
    df = data[(data.Type==pointtype)]
    df = pd.pivot_table(df, columns=["Location","DeviceNum"], index=df.index, values="value")
    df = df.resample('5min').mean()
    return df

In [17]:
data = client.query('SELECT * FROM bmspoint WHERE time > now() - 2d AND time < now() - 1d')
data = pd.DataFrame(data['bmspoint'])
tz_pytz = pytz.timezone('Asia/Singapore')
data = data.tz_convert(tz_pytz)

In [94]:
data[data.Type=='ValvePosition']

,Area,Equip,PointGroup,PointType,SubEquip1,SubEquip2,Type,name,value


## Get CO2

In [91]:
def extract_data_from_influx(timeranges, pointname, filename):
    df_all = pd.DataFrame()
    for timerange in timeranges:
        print "timerange"+str(timerange)
        df = get_data(timerange[1],timerange[0], pointname)
        df_all = pd.concat([df_all, df], axis=0)
    #df = df.tz_convert("None")
    df_all.to_csv(filename)
    return df_all

In [87]:
#co2ex = extract_data_from_influx(dec_feb_timeranges, "CO2", "CO2_Dec_Feb.csv")

In [ ]:
# all_data_co2 = pd.DataFrame()
# for timerange in dec_feb_timeranges:
#     print "timerange"+str(timerange)
#     df = get_data(timerange[1],timerange[0],'CO2Ex')
#     all_data_co2 = pd.concat([all_data_co2_febmar, df], axis=0)
# all_data_co2.to_csv("CO2Ex_Dec_Feb.csv")

In [ ]:
co2ex.head()

In [24]:
# all_data_co2_febmar.resample('H').mean().plot(subplots=True, figsize=(10,30));

## Get BMS Air Flowrate

In [28]:
airflows = extract_data_from_influx(dec_feb_timeranges, "AirFlowrate", "Airflows_Dec_Feb.csv")

timerange(63, 61)
timerange(61, 59)
timerange(59, 57)
timerange(57, 55)
timerange(55, 53)
timerange(21, 19)
timerange(19, 17)
timerange(17, 15)
timerange(15, 13)


In [29]:
airflows.head()

Location,FCU1,FCU2,FCU3,FCU4
DeviceNum,Num0,Num0,Num0,Num0
2016-12-30 13:55:00+08:00,0.0000,37.675001,0.0,-0.037500
2016-12-30 14:00:00+08:00,0.0000,36.650000,0.0,-0.037500
2016-12-30 14:05:00+08:00,0.0000,36.333335,0.0,-0.050000
2016-12-30 14:10:00+08:00,0.0125,36.825001,0.0,-0.050000
2016-12-30 14:15:00+08:00,0.0000,37.133334,0.0,-0.041667


## Coil Valve Position

In [95]:
coilvalve = extract_data_from_influx(dec_feb_timeranges, "ValvePosition", "PCBValve_Dec_Feb.csv")

timerange(73, 71)
timerange(71, 69)
timerange(69, 67)
timerange(67, 65)
timerange(65, 63)
timerange(31, 29)
timerange(29, 27)
timerange(27, 25)
timerange(25, 23)


## Water supply temp

In [31]:
supplywater = extract_data_from_influx(dec_feb_timeranges, "WaterSupplyTemp", "WaterSupplyTemp_Dec_Feb.csv")

timerange(63, 61)
timerange(61, 59)
timerange(59, 57)
timerange(57, 55)
timerange(55, 53)
timerange(21, 19)
timerange(19, 17)
timerange(17, 15)
timerange(15, 13)


# Water flow rates

In [32]:
waterflow = extract_data_from_influx(dec_feb_timeranges, "WaterFlowrate", "WaterFlowRate_Dec_Feb.csv")

timerange(63, 61)
timerange(61, 59)
timerange(59, 57)
timerange(57, 55)
timerange(55, 53)
timerange(21, 19)
timerange(19, 17)
timerange(17, 15)
timerange(15, 13)


## Get BMS Air System Modulation

In [70]:
host='horsebrokedown-clocktower-1.c.influxdb.com' 
port='8086'
login='influxdb'
pw='a873671b790d4523'
dbname = 'siemenstest'

# host='lonepine-76a67992.influxcloud.net'
# port='8086' 
# login='loaddata'
# pw='uwcsearocks88'
# database = 'bms'

client = DataFrameClient(host, port, login, pw, dbname, ssl=True, verify_ssl=True)

In [71]:
def get_data_horse(begin, end, pointtype):
    data = client.query('SELECT * FROM bmspoint WHERE time > now() - '+str(end)+'d AND time < now() - '+str(begin)+'d')
    data = pd.DataFrame(data['bmspoint'])
    tz_pytz = pytz.timezone('Asia/Singapore')
    data = data.tz_convert(tz_pytz)
    df = data[(data.PointType==pointtype)]
    df = pd.pivot_table(df, columns=["name"], index=df.index, values="value")
    df = df.resample('5min').mean()
    return df

In [97]:
def extract_data_from_influx_horse(timeranges, pointname, filename):
    df_all = pd.DataFrame()
    for timerange in timeranges:
        print "timerange"+str(timerange)
        df = get_data_horse(timerange[1],timerange[0], pointname)
        df_all = pd.concat([df_all, df], axis=0)
    df_all.to_csv("./extracted_data/new_space/bulk_data/"+filename)
    return df_all

In [75]:
#fanmodulation = extract_data_from_influx_horse(dec_feb_timeranges, "MdltSu", "FanModulation_Dec_Feb.csv")

In [74]:
fanmodulation.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5189 entries, 2016-12-30 16:15:00+08:00 to 2017-02-18 16:55:00+08:00
Data columns (total 4 columns):
B10_Area3for2_A_Ahu1_DOAS_FanSuEh_MdltSu    5178 non-null float64
B10_Area3for2_A_Ahu2_DOAS_FanSuEh_MdltSu    5178 non-null float64
B10_Area3for2_A_Ahu3_DOAS_FanSuEh_MdltSu    5178 non-null float64
B10_Area3for2_A_Ahu4_DOAS_FanSuEh_MdltSu    5178 non-null float64
dtypes: float64(4)
memory usage: 202.7 KB


## Air system supply flowrates

In [79]:
fanairflow = extract_data_from_influx_horse(dec_feb_timeranges, "FlSu", "AirflowRate_Dec_Feb.csv")

timerange(73, 71)
timerange(71, 69)
timerange(69, 67)
timerange(67, 65)
timerange(65, 63)
timerange(31, 29)
timerange(29, 27)
timerange(27, 25)
timerange(25, 23)


In [82]:
fanairflow.to_csv("./extracted_data/new_space/bulk_data/AirflowRate.csv")